In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
from pathlib import Path
import os

import desispec
from desispec.spectra import Spectra
from desispec.io import read_spectra
from astropy.io import fits

In [ ]:
import logging
logging.getLogger('desispec').setLevel(logging.WARNING)

### Object oriented?

In [51]:
class Target:
    def __init__(self, night:int, tileid:int, spectra:Spectra):
        self.night = night
        self.tileid = tileid
        self.spectra = spectra
        self.petal = spectra.fibermap['PETAL_LOC'][0]
        self.id = spectra.fibermap['TARGETID'][0]

    def get_flux(self):
        return self.spectra.flux

    def get_wave(self):
        return self.spectra.wave

    def set_spectra(self, spectra):
        self.spectra = spectra


class Dataset:
    def __init__(self, release:str, survey:str):
        self.release = release
        self.survey = survey
        self.targets = defaultdict(dict)

    def get_target(self, targetid:int):
        return self.targets[targetid]

    def add_target(self, target:Target):
        self.targets[target.id] = target

    def remove_target(self, target:Target):
        del self.targets[target.id]

In [52]:
data_path = '/Users/valeriatorresgomez/onedrive/tiny_dr1/spectro/redux/iron/tiles/cumulative/10256/20211110/coadd-0-10256-thru20211110.fits'
spectra = desispec.io.read_spectra(data_path)

mask = np.logical_and(spectra.fibermap['COADD_FIBERSTATUS']==0, spectra.fibermap['DESI_TARGET']!=0)
index = np.where(mask)[0]

tgt = np.random.choice(index)
coadd_spec = spectra[tgt]
spectra.fibermap['TARGETID'][0]

INFO:spectra.py:391:read_spectra: iotime 34.974 sec to read coadd-0-10256-thru20211110.fits at 2025-02-21T12:40:53.699373


39627539206898128

### Write data in csv

In [53]:
def create_csv_dataset(data_path, tiles_ids, nights, output_csv_flux, output_csv_wave):
    rows_flux, rows_wave = [], []

    for tile, night in zip(tiles_ids, nights):
        folder = os.path.join(data_path, str(tile), str(night))

        if not os.path.isdir(folder):
            print(f'Foldes {folder} does not exist\n')
            continue
        petal_files = [f for f in os.listdir(folder) if f.startswith('coadd-')]

        if not petal_files:
            print(f'No COADD file in {folder}\n')
            continue

        for petal in petal_files:
            file_path = os.path.join(folder, petal)
            try:
                sp = desispec.io.read_spectra(file_path)
            except Exception as e:
                print(f'Exception reading {file_path}: {e}\n')
                continue
            fibermap = sp.fibermap.to_pandas()
            mask = (fibermap['COADD_FIBERSTATUS'] == 0) & (fibermap['DESI_TARGET'] != 0)
            valid_indices = np.where(mask)[0]
            if valid_indices.size == 0:
                continue

            flux_b = sp.flux['b']
            flux_r = sp.flux['r']
            flux_z = sp.flux['z']
            flux_brz = [np.concatenate([b, r, z]) for b, r, z in zip(flux_b, flux_r, flux_z)]
            for pos in valid_indices:
                target_info = fibermap.iloc[pos]
                rows_flux.append({'TARGETID': target_info.get('TARGETID', None),
                                  'TILEID': target_info.get('TILEID', tile),
                                  'PETAL_LOC': target_info.get('PETAL_LOC', None),
                                  'FLUX_B': flux_b[pos], 'FLUX_R': flux_r[pos],
                                  'FLUX_Z': flux_z[pos], 'FLUX_BRZ': flux_brz[pos]})

            wave_b, wave_r, wave_z = sp.wave['b'], sp.wave['r'], sp.wave['z']
            wave_brz = np.concatenate([wave_b, wave_r, wave_z])
            rows_wave.append({'TILEID': tile, 'NIGHT': night,
                              'PETAL_FILE': petal, 'WAVE_B': wave_b,
                              'WAVE_R': wave_r, 'WAVE_Z': wave_z,
                              'WAVE_BRZ': wave_brz})

    df_flux, df_wave = pd.DataFrame(rows_flux), pd.DataFrame(rows_wave)
    df_flux.to_csv(output_csv_flux, index=False); df_wave.to_csv(output_csv_wave, index=False)
    print(f"Flux data saved in {output_csv_flux}")
    print(f"Wave data saved in {output_csv_wave}")
    return df_flux, df_wave

In [56]:
path = Path('./data')
path.mkdir(parents=True, exist_ok=True)

In [ ]:
# !rclone mount onedrive: ~/onedrive --vfs-cache-mode writes

In [58]:
data_path = '/Users/valeriatorresgomez/onedrive/tiny_dr1/spectro/redux/iron/tiles/cumulative'
tiles_ids = os.listdir(data_path)
nights = [os.listdir(os.path.join(data_path, tiles_ids[i]))[0] for i in range(len(tiles_ids))]

output_csv_flux, output_csv_wave = './data/flux_data.csv', './data/wave_data.csv'
df_flux, df_wave = create_csv_dataset(data_path, tiles_ids, nights, output_csv_flux, output_csv_wave)

INFO:spectra.py:391:read_spectra: iotime 35.933 sec to read coadd-0-10256-thru20211110.fits at 2025-02-21T12:44:02.629668
INFO:spectra.py:391:read_spectra: iotime 35.303 sec to read coadd-1-10256-thru20211110.fits at 2025-02-21T12:44:38.107437
INFO:spectra.py:391:read_spectra: iotime 39.943 sec to read coadd-2-10256-thru20211110.fits at 2025-02-21T12:45:18.869463
INFO:spectra.py:391:read_spectra: iotime 44.953 sec to read coadd-3-10256-thru20211110.fits at 2025-02-21T12:46:06.951517
INFO:spectra.py:391:read_spectra: iotime 37.965 sec to read coadd-4-10256-thru20211110.fits at 2025-02-21T12:46:45.087156
INFO:spectra.py:391:read_spectra: iotime 35.145 sec to read coadd-5-10256-thru20211110.fits at 2025-02-21T12:47:20.824999
INFO:spectra.py:391:read_spectra: iotime 37.853 sec to read coadd-6-10256-thru20211110.fits at 2025-02-21T12:47:59.286612
INFO:spectra.py:391:read_spectra: iotime 32.317 sec to read coadd-7-10256-thru20211110.fits at 2025-02-21T12:48:31.714780
INFO:spectra.py:391:read